In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

np.random.seed(42)
tf.random.set_seed(42)

DATA_PATH = os.path.join(os.getcwd(), 'data', 'breast-cancer-wisconsin.data')

In [2]:
names = [
    'id', 
    'clump_thickness', 
    'cell_size_uniformity', 
    'cell_shape_uniformity',
    'marginal_adhesion',
    'epithelial_cell_size',
    'bare_nuclei',
    'bland_chromatin',
    'normal_nucleoli',
    'mitoses',
    'class'
]

df = pd.read_csv(DATA_PATH, names=names)

# remove NaN values
df = df.replace('?', np.NaN)
df.dropna(inplace=True)

# convert bare_nuclei to int
df['bare_nuclei'] = df['bare_nuclei'].astype(int)

# remove non-feature col
df.drop(columns=['id'], inplace=True)

# convert labels to M=1 and B=0
df['class'] = (df['class'] == 4).astype(int)

df.head()

,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [3]:
# spilt features and labels
X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [4]:
# compute class weights (data is unbalanced)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {
    0: class_weights[0],
    1: class_weights[1],
}
class_weights

{0: 0.7479452054794521, 1: 1.5082872928176796}

In [5]:
# model architecture
model = keras.models.Sequential([
    keras.layers.Input(shape=[9,]),
    keras.layers.Dense(16, activation='selu'),
    keras.layers.Dense(8, activation='selu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train, epochs=50, 
          validation_data=(X_test, y_test), 
          class_weight=class_weights, verbose=True)

Epoch 1/50
18/18 [==============================] - 0s 8ms/step - loss: 1.5859 - accuracy: 0.5971 - val_loss: 0.8417 - val_accuracy: 0.5766
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.7198 - val_loss: 0.5080 - val_accuracy: 0.8102
Epoch 3/50
18/18 [==============================] - 0s 2ms/step - loss: 0.4132 - accuracy: 0.8553 - val_loss: 0.3536 - val_accuracy: 0.8686
Epoch 4/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2933 - accuracy: 0.9103 - val_loss: 0.2760 - val_accuracy: 0.9124
Epoch 5/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2274 - accuracy: 0.9341 - val_loss: 0.2253 - val_accuracy: 0.9343
Epoch 6/50
18/18 [==============================] - 0s 2ms/step - loss: 0.1856 - accuracy: 0.9505 - val_loss: 0.1928 - val_accuracy: 0.9562
Epoch 7/50
18/18 [==============================] - 0s 2ms/step - loss: 0.1578 - accuracy: 0.9579 - val_loss: 0.1796 - val_accuracy: 0.9343
Epoch 8/50
18/18 [==

In [7]:
y_test_pred = model.predict(X_test).reshape(-1,) > 0.5

In [8]:
confusion_matrix(y_test, y_test_pred)

array([[78,  1],
       [ 3, 55]])

## DICE

In [9]:
import dice_ml

In [16]:
query_instance = X_test.iloc[50].to_dict() # class = 0
query_instance

0

In [11]:
continuous_features = list(df.columns)[:-1]
continuous_features

['clump_thickness',
 'cell_size_uniformity',
 'cell_shape_uniformity',
 'marginal_adhesion',
 'epithelial_cell_size',
 'bare_nuclei',
 'bland_chromatin',
 'normal_nucleoli',
 'mitoses']

In [28]:
d = dice_ml.Data(dataframe=df, continuous_features=continuous_features, outcome_name='class')
m = dice_ml.Model(model=model, backend='TF2')
exp = dice_ml.Dice(d,m)

In [29]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=1)

Diverse Counterfactuals found! total time taken: 00 min 21 sec


In [30]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,2.0,2.0,2.0,1.0,1.0,1.0,7.0,1.0,1.0,0.000794



Diverse Counterfactual set (new outcome : 1)


,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
